<a href="https://colab.research.google.com/github/carolgirafa/Pluviometric_Study_RJ/blob/main/pluviometric_study_RJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pluviometric study in the city of Rio de Janeiro

Even though Rio is famous for its sunny days, beaches and 40 Celsius degrees weather, it's also a humid city.
The average annual relative humidity is 79.1%.
Today we'll study a dataset from Rio's Town Hall.

Questions:

1.   Does it rain more nowadays like people are saying?
2.   Do the "March waters" really exist?





## Importing

In [ ]:
!pip install basedosdados "shapely<2"

In [2]:
import basedosdados as bd

In [99]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [4]:
# Importing full dataset
df_full = bd.read_sql("SELECT * FROM `datario.meio_ambiente_clima.taxa_precipitacao_alertario` LIMIT 30000000" , billing_project_id = "pluviometric-study-rj" )

Downloading: 100%|██████████| 28713199/28713199 [47:19<00:00, 10113.15rows/s]


## Data Cleaning

In [134]:
# Checking the data
df_full.info()

# Checking for NaN
df_full.isna().sum()/df_full.count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28713199 entries, 0 to 28713198
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   primary_key             object 
 1   id_estacao              object 
 2   acumulado_chuva_15_min  float64
 3   acumulado_chuva_1_h     float64
 4   acumulado_chuva_4_h     float64
 5   acumulado_chuva_24_h    float64
 6   acumulado_chuva_96_h    float64
 7   horario                 dbtime 
 8   data_particao           dbdate 
dtypes: dbdate(1), dbtime(1), float64(5), object(2)
memory usage: 1.9+ GB


primary_key               0.000000
id_estacao                0.000000
acumulado_chuva_15_min    0.000540
acumulado_chuva_1_h       0.000716
acumulado_chuva_4_h       0.001025
acumulado_chuva_24_h      0.001690
acumulado_chuva_96_h      0.057444
horario                   0.000042
data_particao             0.000000
dtype: float64

The NaN percentages are low and allow us to drop the data

In [135]:
#Removing rows with NaN in the chosen column

df_full[df_full['acumulado_chuva_15_min'].notna()]

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao
0,1_1997-08-02 05:03:20,1,0.0,0.0,0.0,0.0,0.0,05:03:20,1997-08-02
1,1_1997-08-04 06:03:20,1,0.0,0.0,0.0,0.0,0.0,06:03:20,1997-08-04
2,1_1997-08-04 19:18:20,1,0.0,0.0,0.0,0.0,0.0,19:18:20,1997-08-04
3,1_1997-08-04 14:48:20,1,0.0,0.0,0.0,0.0,0.0,14:48:20,1997-08-04
4,1_1997-08-04 19:48:20,1,0.0,0.0,0.0,0.0,0.0,19:48:20,1997-08-04
...,...,...,...,...,...,...,...,...,...
28713194,13_2006-03-06 22:31:00,13,0.0,0.0,0.0,1.6,1.6,22:31:00,2006-03-06
28713195,10_2006-03-27 19:47:40,10,0.0,0.0,0.0,2.6,24.6,19:47:40,2006-03-27
28713196,31_2006-03-06 07:49:40,31,2.6,2.6,2.6,2.6,2.6,07:49:40,2006-03-06
28713197,24_2006-03-31 08:38:40,24,0.0,0.0,0.0,2.6,2.6,08:38:40,2006-03-31


In [136]:
# Group by date to have the aggregate rain per day

df_daily = df_full.groupby(['data_particao'])\
                  .agg({'acumulado_chuva_15_min':'sum'})\
                  .reset_index()

## Overview

In [137]:
# Creating a line plot with plotly
fig1 = px.line(df_daily, x='data_particao', y='acumulado_chuva_15_min',
              hover_data={'data_particao': '|%B %d, %Y'},
              title='Rain behaviour overview',
              )

# Creating range options for better visualization
fig1.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Changing x_label and y_label
fig1.update_layout(
    xaxis_title='Date',
    yaxis_title='Rain mm'
)
fig1.show()

Looking at this plot we can notice the rainfall ammount has not increased over the years, like our previous assumption.

In addition, we can notice that the rainiest day of all was 02/26/2016.

## Preparing data for further time based analysis

In [138]:
#Transforming the data_particao column in datetime
df_daily['data_particao'] = pd.to_datetime(df_daily['data_particao'])

#Creating a month column
df_daily['month'] = pd.Series(df_daily['data_particao']).dt.month

#Creating a year column
df_daily['year'] = pd.Series(df_daily['data_particao']).dt.year

## Year based analysis

In [139]:
# Aggregating per year
df_year = df_daily.groupby('year')\
                  .agg({'acumulado_chuva_15_min':'sum'})\
                  .reset_index()

Top 5 rainy years

In [140]:
# Finding the top 5 years with most rain

df_year.sort_values(by='acumulado_chuva_15_min',ascending=False).head(5)

,year,acumulado_chuva_15_min
12,2009,64651.4
1,1998,64289.6
6,2003,64208.6
13,2010,61409.0
16,2013,60464.2


In [141]:
# Creating a bar plot
fig2 = px.bar(df_year, 
              x='year',
              y='acumulado_chuva_15_min',
              title='Rain through the years',
              )

# Changing x_label and y_label
fig2.update_layout(
    xaxis_title='Years',
    yaxis_title='Rain mm'
)
fig2.show()

## Month based analysis

In [142]:
# Aggregating per month
df_month = df_daily.groupby(['year' ,'month'])\
                  .agg({'acumulado_chuva_15_min':'sum'})\
                  .reset_index()

In [143]:
# Calculating month average rainfall
df_month_avg = df_month.groupby('month')\
                       .agg({'acumulado_chuva_15_min':'mean'})\
                       .reset_index()

Top 3 rainy months

In [144]:
df_month_avg.sort_values(by='acumulado_chuva_15_min',ascending=False).head(3)

,month,acumulado_chuva_15_min
1,2,6472.850000
9,10,6166.768000
0,1,5181.188462


In [147]:
# Creating a figure with plotly.graph_objects
fig3 = go.Figure()

# Adding the historical data as scatter plot
fig3.add_trace(go.Scatter(x=df_month['month'], 
                          y=df_month['acumulado_chuva_15_min'], 
                          mode='markers', 
                          name='Rainfall data from all the year'))

# Adding the month average for comparison
fig3.add_trace(go.Scatter(x=df_month_avg['month'], 
                          y=df_month_avg['acumulado_chuva_15_min'], 
                          mode='lines', 
                          name='Average Rainfall in the month'))

# Changing x_label, y_label and legend
fig3.update_layout(
    title='Monthly rain behaviour',
    xaxis_title='Years', 
    yaxis_title='Rain mm',
    legend=dict(
    orientation='h',
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.6
  )
)

fig3.show()

## Conclusion

Finally, we can conclude that:



1.   Does it rain more nowadays like people are saying? **No**
2.   Do the "March waters" really exist? **Also no, they should be called February Waters**
